## Chapter2 오토인코더와 생성 학습
- 이장에서 다음 내용을 다룹니다.
- 잠재 공간으로 데이터를 인코더하고(차원 축소) 그다음 차원 확장하기
- 변이형 오토인코더를 보면서 생성 모델링의 어려움을 이해한다.
- 케라스와 오토 인코더를 이용한 손글씨 숫자 생성하기
- 오토 인코더의 한계와 GAN의 필요성 이해하기

### 생성 모델은 대부분 사람들에게 새로운 분야입니다.
- 오토인코더는 GAN의 원조 격에 가장 가깝고 자료와 연구 결과가 풍부하다.
### 생성 모델은 매우 도전적이다.
- 오토인코더는 여러가지 면에서 흔히 배우는 모델(추후 다루게 될 명시적으로 목적 함수를 가진 모델)과 더 가깝지만 샘플의 품질을 평가하는 어려움 등 GAN이 마주하는 난점을 오터인코더에서도 발견할 수 있다.
### 생성 모델은 오늘날 중요한 분야이다.
- 오토인코더는 나름 용도가 있다. 오토인코더는 여전히 활발히 연구가 진행되는 분야이고 몇몇 분야에서 최첨단의 성과를 내며 많은 GAN구조에 사용되고있다.

## 2.1 생성 모델링 시작하기
- 생성하고 싶은 이미지를 먼저 계획한 다음 변환가정 끝에서 이미지를 얻는다. 이것이 가장 간단한 비정형 형태의 생성 모델링이다.
- 특정한 계획 z가(0~9까지의 숫자)생성된 샘플인 $x^{*}$에 도달하려고 한다. 이상적으로 이 $x^{*}$은 실제 샘플인 x와 다를바 없어 보인다.
- z는 잠재공간(latent space)에 위치하며 항상 같은 결과값 $x^{*}$을 얻지 않도록 도와준다. 잠재 공간은 학습된 표현방식이다. 즉 사람이 생각하는 방식과 비슷하다.
- 다른 모델은 같은 데이터에서 다른 잠재표현(latent representation)을 학습할 것이다.

- chapter 1 에 낳온 랜덤한 잡음 벡터는 잠재 공간에서 얻은 샘플이다.
- 잠재 공간은 데이터 포인트를 더 간단하게 표현한 숨겨진 표현 방식이다. 즉, 저차원공간이다.
- 데이터 포인트의 좋은 잠재 표현은 이 공간 속에서 유사한 것들끼리 묶는데 도움이 된다.
- 오토인코더에서 잠재(latent)의 의미가 무엇인지 알아보자.
- 그리고 생성된 샘플에 어떻게 영향을 미치는지 알아보자.

## 2.2 오토인코더의 동작 방식

In [2]:
from tensorflow.keras.layers import Input,Dense,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.datasets import mnist
import numpy as np

In [3]:
batch_size = 100
original_dim = 784 # h*L
latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0


In [7]:
def sampling(args: tuple):
    z_mean,z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0],latent_dim),mean=0.,stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2)*epsilon

In [8]:
x = Input(shape = (original_dim,),name = "input")
h = Dense(intermediate_dim, activation='relu', name = 'encoding')(x)
z_mean = Dense(latent_dim,name="mean")(h)
z_log_var = Dense(latent_dim,name="log-variance")(h)
z = Lambda(sampling,output_shape = (latent_dim,))([z_mean,z_log_var])

encoder = Model(x,[z_mean,z_log_var,z],name="encoder")

In [9]:
input_decoder = Input(shape = (latent_dim,),name='decoder_input')
decoder_h = Dense(intermediate_dim,activation = 'relu',name='decoder_h')(input_decoder)
x_decoded = Dense(original_dim,activation='sigmoid',name="flat_decoded")(decoder_h)
decoder = Model(input_decoder,x_decoded,name="decoder")

In [11]:
output_combined = decoder(encoder(x)[2])
vae = Model(x,output_combined)
vae.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               201988    
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               202256    
                                                                 
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [12]:
kl_loss = -0.5*K.sum(
    1+z_log_var-K.exp(z_log_var)-K.square(z_mean),
    axis=-1
)

vae.add_loss(K.mean(kl_loss)/784.)
vae.compile(optimizer="rmsprop",loss="binary_crossentropy")

In [18]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.
x_train = x_train.reshape((len(x_train),np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test),np.prod(x_test.shape[1:])))

In [19]:
vae.fit(x_train,x_train,shuffle="True",epochs=epochs,batch_size=batch_size)

Epoch 1/50


2022-06-11 18:12:40.602691: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-11 18:12:40.814342: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 7s 8ms/step - loss: 0.2454
Epoch 2/50
600/600 [==============================] - 5s 8ms/step - loss: 0.2188
Epoch 3/50
600/600 [==============================] - 5s 8ms/step - loss: 0.2136
Epoch 4/50
600/600 [==============================] - 5s 8ms/step - loss: 0.2102
Epoch 5/50
600/600 [==============================] - 6s 9ms/step - loss: 0.2077
Epoch 6/50
600/600 [==============================] - 5s 9ms/step - loss: 0.2062
Epoch 7/50
600/600 [==============================] - 5s 9ms/step - loss: 0.2051
Epoch 8/50
600/600 [==============================] - 5s 8ms/step - loss: 0.2041
Epoch 9/50
600/600 [==============================] - 5s 9ms/step - loss: 0.2032
Epoch 10/50
600/600 [==============================] - 5s 8ms/step - loss: 0.2025
Epoch 11/50
600/600 [==============================] - 5s 9ms/step - loss: 0.2018
Epoch 12/50
600/600 [==============================] - 5s 9ms/step - loss: 0.2012
Epoch 13/50
600/600 [===============